In [ ]:
%cd /home/praval/piper/src/python

In [ ]:
%pwd

### Do this for getting the file to do right location 

In [ ]:
# cp /home/praval/piper/src/python/training_dir/lightning_logs/version_31/checkpoints/epoch=3187-step=3722762.ckpt /home/praval/piper/src/python

### Export Model to ONNX Format

In [ ]:
# Exporting a Model
!python3 -m piper_train.export_onnx \
   ckpts/male_telugu/epoch=2171-step=1379438.ckpt \
    testing/model_2171_te.onnx
!cp training_dir_telugu/config.json \
   testing/model_2171_te.onnx.json

### Test Examples

In [ ]:
!echo 'प्रिय महोदय' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo 'hello world' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo ' जय श्री राम, वास्तविक समय की हिंदी टी टी एस दुनिया में आपका स्वागत है। यह पाइपर द्वारा संचालित है |' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo ' अज्ञानी क्षत्रिय झल्लाकर ढक्कन में दूध उबाल रहा है, जबकि भोली स्त्री फूस की झोपड़ी में बैठी पढाई कर रही है|' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo ' एक ज्ञानी क्षत्रिय ने झील के किनारे बैठकर स्वादिष्ट फल, दूध और घी से भरी थाली में भिक्षुक को भोजन परोसा |' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo 'प्लीज़ आप बाइक को सही से पार्क करें' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo ' द कुइक्क ब्राउन फोक्स जम्प्स ओवर द लेजी डोग, quick brown fox jumps over the lazy dog ' | piper -m testing/model.onnx --output_file test.wav

In [ ]:
!echo ' जय हनुमान ज्ञान गुण सागर जय कपीस तिहु लोक उजागर|' | piper -m testing/model.onnx --output_file test.wav